## Import necessary libraries

In [30]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

## Read cleaned data files

### Read cleaned jobs listings file and save in dataframe

In [31]:
cleaned_jobs_listings_df = pd.read_csv("cleaned_jobs_listings.csv")

### Check columns info in jobs listings dataframe

In [32]:
cleaned_jobs_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_title                3909 non-null   object 
 1   salary_estimate          3909 non-null   object 
 2   job_description          3909 non-null   object 
 3   rating                   3909 non-null   float64
 4   company_name             3909 non-null   object 
 5   location                 3909 non-null   object 
 6   headquarters             3909 non-null   object 
 7   size                     3909 non-null   object 
 8   founded                  3909 non-null   int64  
 9   type_of_ownership        3909 non-null   object 
 10  industry                 3909 non-null   object 
 11  sector                   3909 non-null   object 
 12  revenue                  3909 non-null   object 
 13  competitors              3909 non-null   object 
 14  easy_apply              

## Data Preprocessing

### Define stopwords that appear in both articles and jobs listings

In [33]:
new_common_stopwords = ['job', 'skill', 'experience', 'team', 'data', 'science', 'use', 'like', 'business', 'work', 'ability', 'let', 'example'
                'need', 'new', 'user', 'opportunity', 'candidate', 'provide', 'company', 'think', 'thing', 'function', 'learning']

In [34]:
# append the defined common stopwords with those provided by default in gensim stopwords
new_common_stopwords = STOPWORDS.union(set(new_common_stopwords))

In [35]:
# append special stopwords in articles to default and common stopwords
jobs_listings_stopwords = new_common_stopwords.union(set(['year']))

In [36]:
jobs_listings_stopwords  = list(frozenset(jobs_listings_stopwords))

In [37]:
index = jobs_listings_stopwords.index('year')
index

105

### Method to preprocess data for articles

In [38]:
# lemmatize = words change to noun
# stemmed = words reduced to root form
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='n')

# Convert a document into a list of tokens.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in jobs_listings_stopwords and len(token) > 3:
            result.append(lemmatize(token))
    return result

In [39]:
preprocessed_jobs_listings = cleaned_jobs_listings_df['cleaned_job_description'].fillna('').astype(str).map(preprocess)

In [40]:
# implements the concept of a Dictionary – a mapping between words and their integer ids.
jobs_listings_dictionary = corpora.Dictionary(preprocessed_jobs_listings)

In [41]:
# Convert into the bag-of-words (BoW) format
jobs_listings_bow_corpus = [jobs_listings_dictionary.doc2bow(text) for text in preprocessed_jobs_listings]

In [42]:
# Convert into TF-IDF format
jobs_listings_tfidf = models.TfidfModel(jobs_listings_bow_corpus)
jobs_listings_tfidf_corpus = jobs_listings_tfidf[jobs_listings_bow_corpus]

## LSA Model

### Result using LSA model + Bag of words

In [43]:
jobs_listings_bow_lsamodel = LsiModel(jobs_listings_bow_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [44]:
jobs_listings_bow_lsamodel.print_topics(num_words=7)

[(0,
  '0.173*"development" + 0.166*"year" + 0.158*"analysis" + 0.145*"skill" + 0.143*"technology" + 0.136*"information" + 0.135*"solution"'),
 (1,
  '0.240*"analytics" + -0.196*"information" + -0.175*"research" + 0.170*"product" + -0.168*"laboratory" + 0.158*"model" + 0.148*"solution"'),
 (2,
  '-0.287*"quantum" + -0.203*"world" + 0.156*"analysis" + -0.156*"employee" + 0.144*"analytics" + 0.144*"equivalent" + 0.137*"project"'),
 (3,
  '-0.319*"equivalent" + -0.197*"education" + 0.193*"research" + 0.182*"laboratory" + -0.172*"band" + 0.171*"clinical" + 0.158*"product"'),
 (4,
  '0.332*"research" + -0.241*"information" + 0.232*"scientist" + 0.181*"equivalent" + 0.143*"machine" + 0.137*"education" + -0.134*"application"'),
 (5,
  '-0.358*"development" + 0.210*"model" + -0.209*"process" + 0.203*"machine" + -0.180*"project" + 0.149*"employee" + -0.144*"support"'),
 (6,
  '-0.381*"research" + 0.301*"laboratory" + 0.204*"medical" + -0.192*"analysis" + 0.187*"service" + -0.167*"project" + -0.

### Result using LSA model + TF-IDF

In [45]:
jobs_listings_tfidf_lsamodel = LsiModel(jobs_listings_tfidf_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [46]:
jobs_listings_tfidf_lsamodel.print_topics(num_words=7)

[(0,
  '0.094*"analytics" + 0.090*"machine" + 0.086*"model" + 0.083*"product" + 0.081*"client" + 0.080*"customer" + 0.078*"solution"'),
 (1,
  '0.613*"quantum" + 0.191*"qiskit" + 0.177*"progressive" + 0.112*"world" + 0.101*"quantumenabled" + 0.101*"numerics" + 0.098*"reinventing"'),
 (2,
  '-0.288*"laboratory" + -0.191*"clinical" + -0.154*"cell" + -0.115*"scientific" + -0.114*"assay" + 0.111*"machine" + -0.107*"biology"'),
 (3,
  '-0.181*"locate" + -0.180*"analyze" + -0.176*"printout" + -0.172*"databasesdata" + -0.159*"qualityacquire" + -0.159*"setsfilter" + -0.158*"systemsidentify"'),
 (4,
  '-0.168*"university" + 0.162*"cell" + -0.139*"security" + -0.133*"clery" + 0.129*"clinical" + -0.120*"investigation" + 0.109*"assay"'),
 (5,
  '-0.730*"tutor" + -0.490*"tutoring" + -0.175*"student" + -0.132*"choose" + -0.124*"session" + -0.086*"schedule" + -0.082*"tutoringonline"'),
 (6,
  '-0.159*"pipeline" + -0.142*"cloud" + -0.134*"cell" + -0.133*"laboratory" + -0.133*"spark" + -0.127*"azure" +

## LDA Model

### Result using LDA model + Bag of words

In [47]:
jobs_listings_bow_ldamodel = gensim.models.LdaMulticore(jobs_listings_bow_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [48]:
jobs_listings_bow_ldamodel.print_topics(num_words=7)

[(0,
  '0.006*"year" + 0.006*"analysis" + 0.005*"technology" + 0.005*"solution" + 0.005*"model" + 0.005*"development" + 0.005*"product"'),
 (1,
  '0.007*"skill" + 0.006*"analysis" + 0.006*"scientist" + 0.006*"year" + 0.005*"research" + 0.005*"project" + 0.005*"product"'),
 (2,
  '0.008*"year" + 0.005*"status" + 0.005*"skill" + 0.005*"analysis" + 0.005*"project" + 0.005*"machine" + 0.005*"information"'),
 (3,
  '0.006*"analysis" + 0.006*"skill" + 0.005*"product" + 0.005*"year" + 0.005*"model" + 0.005*"knowledge" + 0.004*"management"'),
 (4,
  '0.007*"development" + 0.007*"analytics" + 0.005*"skill" + 0.005*"year" + 0.005*"analysis" + 0.005*"product" + 0.005*"working"'),
 (5,
  '0.005*"year" + 0.005*"development" + 0.005*"engineering" + 0.004*"support" + 0.004*"skill" + 0.004*"position" + 0.004*"employee"'),
 (6,
  '0.007*"research" + 0.006*"development" + 0.005*"skill" + 0.005*"project" + 0.005*"year" + 0.005*"product" + 0.004*"analysis"'),
 (7,
  '0.006*"including" + 0.005*"development

In [49]:
pyLDAvis.gensim_models.prepare(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.008384 -0.001726       1        1  16.307950
2      0.001499 -0.002439       2        1  15.217299
8      0.004638  0.001305       3        1  13.571796
9      0.007403  0.002394       4        1  13.145766
0      0.007433  0.002369       5        1  11.352101
6     -0.005264 -0.006780       6        1   9.806535
7      0.000371 -0.004166       7        1   9.799940
1     -0.009637 -0.010797       8        1   4.731226
5     -0.015379  0.010821       9        1   3.415041
3      0.000552  0.009019      10        1   2.652347, topic_info=            Term         Freq        Total Category  logprob  loglift
293        skill  4745.000000  4745.000000  Default  30.0000  30.0000
7       analysis  5030.000000  5030.000000  Default  29.0000  29.0000
140     research  3771.000000  3771.000000  Default  28.0000  28.0000
148    scientist  3271.000000  3271.000000  Default  27.0000  27.0000
130      product  4347.000000  4347.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
501    including    88.742289  4027.442984  Topic10  -5.6783  -0.1854
301   technology    92.732690  4533.684306  Topic10  -5.6343  -0.2598
1108     service    84.104980  3768.741301  Topic10  -5.7319  -0.1727
417      process    76.455900  3546.308821  Topic10  -5.8273  -0.2072
199    analytics    74.229226  4236.015896  Topic10  -5.8568  -0.4145

[859 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
12516      1  0.230586  accuracyadept
12516      4  0.038431  accuracyadept
12516      5  0.269017  accuracyadept
12516      6  0.307448  accuracyadept
12516      8  0.076862  accuracyadept
...      ...       ...            ...
7373       4  0.276086    âexperience
7373       9  0.276086    âexperience
7376       2  0.277175        âshould
7376       4  0.277175        âshould
7376       9  0.277175        âshould

[2229 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 9, 10, 1, 7, 8, 2, 6, 4])

### Result using LDA model + TF-IDF

In [52]:
jobs_listings_tfidf_ldamodel = gensim.models.LdaMulticore(jobs_listings_tfidf_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [53]:
jobs_listings_tfidf_ldamodel.print_topics(num_words=7)

[(0,
  '0.001*"product" + 0.001*"analytics" + 0.001*"research" + 0.001*"analysis" + 0.001*"project" + 0.001*"machine" + 0.001*"process"'),
 (1,
  '0.001*"quantum" + 0.001*"model" + 0.001*"research" + 0.001*"machine" + 0.001*"customer" + 0.001*"client" + 0.001*"analytics"'),
 (2,
  '0.001*"tutor" + 0.001*"analytics" + 0.001*"tutoring" + 0.001*"analyst" + 0.001*"machine" + 0.001*"quantum" + 0.001*"database"'),
 (3,
  '0.001*"customer" + 0.001*"machine" + 0.001*"analytics" + 0.001*"laboratory" + 0.001*"clinical" + 0.001*"model" + 0.001*"statistical"'),
 (4,
  '0.001*"client" + 0.001*"product" + 0.001*"model" + 0.001*"analytics" + 0.001*"machine" + 0.001*"research" + 0.001*"technology"'),
 (5,
  '0.001*"machine" + 0.001*"analytics" + 0.001*"client" + 0.001*"research" + 0.001*"model" + 0.001*"database" + 0.001*"health"'),
 (6,
  '0.001*"client" + 0.001*"machine" + 0.001*"customer" + 0.001*"analytics" + 0.001*"model" + 0.001*"solution" + 0.001*"product"'),
 (7,
  '0.001*"analytics" + 0.001*"

In [54]:
pyLDAvis.gensim_models.prepare(jobs_listings_tfidf_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.018659  0.006321       1        1  38.111661
8      0.004759 -0.008361       2        1  11.963787
5      0.000354  0.000969       3        1  10.776076
1     -0.002387  0.017597       4        1   8.669224
0      0.002488 -0.001884       5        1   8.654529
7     -0.000058 -0.003154       6        1   5.518303
3      0.001640 -0.003232       7        1   5.449132
4     -0.003492 -0.009606       8        1   5.447385
6     -0.004924 -0.003071       9        1   4.080459
2     -0.017039  0.004421      10        1   1.329444, topic_info=             Term        Freq       Total Category  logprob  loglift
199     analytics  739.000000  739.000000  Default  30.0000  30.0000
98        machine  753.000000  753.000000  Default  29.0000  29.0000
8239      quantum  269.000000  269.000000  Default  28.0000  28.0000
340        client  655.000000  655.000000  Default  27.0000  27.0000
35       customer  625.000000  625.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
603         model    6.631557  725.431613  Topic10  -7.5815  -0.3745
45    development    6.238473  573.830525  Topic10  -7.6426  -0.2012
35       customer    6.248886  625.240011  Topic10  -7.6409  -0.2853
106      modeling    5.876943  533.642178  Topic10  -7.7023  -0.1883
131       project    5.835118  558.349462  Topic10  -7.7094  -0.2407

[793 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
18212      1  0.124248   abbott
18212      2  0.155310   abbott
18212      3  0.403807   abbott
18212      4  0.031062   abbott
18212      5  0.124248   abbott
...      ...       ...      ...
7376       4  0.072481  âshould
7376       5  0.072481  âshould
7376       6  0.072481  âshould
7376       7  0.072481  âshould
7376       8  0.289923  âshould

[4430 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 6, 2, 1, 8, 4, 5, 7, 3])